In [1]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility, db
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
connections.connect(
    alias="default",
    host = os.getenv("MILVUS_HOST"),
    port = 19530,
    user=os.getenv("MILVUS_USER"),
    password=os.getenv("MILVUS_PASSWORD"),
)

In [3]:
# db.create_database("JARVIS")

In [4]:
db.using_database("JARVIS")

In [5]:
utility.list_collections()

['calculator']

In [6]:
conversation_id = FieldSchema(name="conversation_id", dtype=DataType.INT64, is_primary=True, auto_id=True, description="Unique id for each conversation")
embeding = FieldSchema(name="embeding", dtype=DataType.FLOAT_VECTOR, dim=1024, description="Embedings generated using UAE_v2")
conversation = FieldSchema(name="conversation", dtype=DataType.JSON, description="conversation data")

schema = CollectionSchema(fields=[conversation_id, embeding, conversation], description="Conversations for using Calculator")

collection = Collection(name="calculator", schema=schema)

In [7]:
collection.create_index(
    field_name="embeding", 
    index_params={
        "index_type": "IVF_FLAT",
        "metric_type": "COSINE",
        "params": {"nlist": 128}
    }
)

Status(code=0, message=)

In [8]:
collection.load()

In [100]:
results = collection.query_iterator(
    batch_size=5,
    output_fields=["conversation"],
    expr="conversation_id >0",
)

In [107]:
conversation = results.next()
conversation[0]["conversation"]["data"]

'USER: I need help with a recipe, I need to double it.\nASSISTANT: I can help you double a recipe. Provide me with the current measurements. </s>\nUSER: The recipe asks for three eggs, four cups of flour and two cups of sugar.\nASSISTANT: The doubled recipe will require <calculator> 3*2 </s> 6 </calculator> six eggs, <calculator> 4*2 </s> 8 </calculator> eight cups of flour and <calculator> 2*2 </s> 4 </calculator> four cups of sugar. </s>'